In [11]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [12]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json') as f:
    bigtable=json.load(f)
with open('D:/Data/JsonData/TainanFood/final_tainan.json') as f:
    rawdata=json.load(f)
with open('D:/Data/JsonData/TainanFood/TainanGoodDien_1.1.json') as f:
    gooddien=json.load(f) 

In [13]:
bigtable[0]

{'Ncomment': 26,
 'TF_IDF': {'一下': 0.0005058619310472912,
  '一個': 0.0006763427417325443,
  '一口': 0.000902388368012658,
  '一家': 0.0008383533159246719,
  '一張': 0.0009772699783033602,
  '一杯': 0.0007763227846572202,
  '一次': 0.0007700843326304692,
  '一碗': 0.0031732457338551666,
  '一種': 0.0006668078310359679,
  '一起': 0.0008712465982758182,
  '一間': 0.0008375586857847692,
  '一點': 0.0010014197611782284,
  '三種': 0.0007760097055008344,
  '上次': 0.0021988574511825607,
  '上面': 0.000604514746360345,
  '下': 0.00012088803629924028,
  '下次': 0.0006209773348036758,
  '不到': 0.0004732557107079696,
  '不用': 0.0006399935237685169,
  '不錯': 0.0003006029692292721,
  '中午': 0.0007751981504600589,
  '之後': 0.0007806238562467455,
  '二樓': 0.0011710565380702723,
  '享用': 0.0006151714242347759,
  '人': 0.0009087926521433939,
  '人潮': 0.0024724144797953255,
  '今天': 0.00044843015836489036,
  '介紹': 0.000613901062466487,
  '以上': 0.0007279636511091276,
  '以前': 0.0009350987128488293,
  '位置': 0.0010154508997988877,
  '來到': 0.00317

In [15]:
rawdata[0]['comment'][0]['content']

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/'

In [46]:
dienopen=[]
for dien in rawdata:
    if dien['business_hours']!=None:
        t=dien['business_hours'].split("~")
    else:
        t=['11:00','19:00']
    dienopen.append(t)
dienopen

[['09:00', '23:00'],
 ['08:00', '22:00'],
 ['05:00', '12:00'],
 ['06:00', '16:00'],
 ['11:00', '21:00'],
 ['11:00', '14:00'],
 ['11:30', '14:30'],
 ['11:00', '23:00'],
 ['08:00', '21:00'],
 ['07:00', '19:30'],
 ['11:00', '19:00'],
 ['00:00', '02:00'],
 ['08:00', '18:00'],
 ['11:00', '19:00'],
 ['11:00', '19:00'],
 ['11:00', '15:00'],
 ['10:00', '22:00'],
 ['12:00', '19:00'],
 ['11:00', '19:00'],
 ['11:00', '15:00'],
 ['11:00', '21:00'],
 ['08:30', '22:00'],
 ['11:00', '19:00'],
 ['16:30', '02:00'],
 ['11:00', '22:00'],
 ['10:00', '21:00'],
 ['11:00', '19:00'],
 ['09:00', '18:00'],
 ['11:00', '18:00'],
 ['00:00', '01:00'],
 ['12:00', '15:00'],
 ['07:30', '18:00'],
 ['07:40', '22:30'],
 ['10:00', '19:00'],
 ['10:30', '22:00'],
 ['11:00', '19:00'],
 ['06:00', '22:00'],
 ['13:00', '21:00'],
 ['11:00', '18:00'],
 ['09:00', '17:00'],
 ['10:00', '23:00'],
 ['09:00', '21:00'],
 ['11:00', '22:00'],
 ['09:30', '13:30'],
 ['11:00', '21:00'],
 ['11:00', '23:00'],
 ['10:00', '21:00'],
 ['00:00', '0

In [47]:
with open("D:\Data\csvtest\style.csv") as f:
    mydict=f.read()
synonyms=[{word:line.split(",")[0] for word in line.split(",")[0:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]
pinsynonymsdict

{'上海菜(江浙菜)': '異國料理',
 '下午茶': '甜點(合併飲品)',
 '中式': '小吃',
 '中式早餐': '早午餐',
 '中式糕餅': '甜點(合併飲品)',
 '休閒零食': '甜點(合併飲品)',
 '俄羅斯料理': '異國料理',
 '其他亞洲料理': '異國料理',
 '其他小吃': '小吃',
 '其他日式料理': '日式',
 '其他異國料理': '異國料理',
 '其他美食': '小吃',
 '其他速食': '異國料理',
 '其他鍋類': '鍋類',
 '其他類型buffet自助餐': '異國料理',
 '其他類型早餐': '早午餐',
 '其它中式料理': '小吃',
 '其它主題餐廳': '異國料理',
 '冰淇淋': '甜點(合併飲品)',
 '刈包': '小吃',
 '剉冰、豆花': '甜點(合併飲品)',
 '動漫主題餐廳': '異國料理',
 '北京菜': '異國料理',
 '印度料理': '異國料理',
 '台菜餐廳': '小吃',
 '咖啡專賣': '飲品',
 '啤酒屋': '酒吧',
 '四川菜': '異國料理',
 '墨西哥料理': '異國料理',
 '壽喜燒': '日式',
 '客家菜': '小吃',
 '寵物餐廳': '異國料理',
 '小吃': '小吃',
 '居酒屋': '酒吧',
 '山產野菜餐廳': '小吃',
 '巧克力專賣': '甜點(合併飲品)',
 '希臘料理': '異國料理',
 '德式料理': '異國料理',
 '情境主題餐廳': '異國料理',
 '懷石料理': '日式',
 '懷舊主題': '小吃',
 '披薩': '異國料理',
 '文字燒、大阪燒': '日式',
 '日式': '日式',
 '日式燒肉': '日式',
 '日式豬排專賣': '日式',
 '日式麵食專賣': '日式',
 '早午餐': '早午餐',
 '星馬料理': '異國料理',
 '景觀餐廳': '酒吧',
 '果昔': '飲品',
 '果汁': '飲品',
 '水煎包、生煎包': '小吃',
 '法式料理': '異國料理',
 '泰式料理': '異國料理',
 '海鮮餐廳': '小吃',
 '涮涮鍋': '鍋類',
 '湘菜(湖南菜)': '異國料理',
 '溫泉餐廳': '小吃',
 '溫體牛': '小

In [48]:
#style需轉換 新增opentime closetime
bigtable[0].keys()

dict_keys(['name', 'style', 'Ncomment', 'contentcut', 'wordCount', 'TF_IDF', 'pinCountMaxper5', 'score', 'featurescores', 'tags', 'introduction', 'coordinate'])

In [49]:
#變更bigtablestyle
for dien in bigtable:
    if dien['style'] in pinsynonymsdict:
        dien['style']=pinsynonymsdict[dien['style']]
    else:
        dien['style']='小吃'

In [50]:
[dien['style'] for dien in bigtable]

['甜點(合併飲品)',
 '甜點(合併飲品)',
 '小吃',
 '小吃',
 '甜點(合併飲品)',
 '異國料理',
 '異國料理',
 '甜點(合併飲品)',
 '甜點(合併飲品)',
 '小吃',
 '飲品',
 '小吃',
 '小吃',
 '甜點(合併飲品)',
 '甜點(合併飲品)',
 '異國料理',
 '小吃',
 '甜點(合併飲品)',
 '異國料理',
 '異國料理',
 '甜點(合併飲品)',
 '小吃',
 '小吃',
 '小吃',
 '甜點(合併飲品)',
 '小吃',
 '甜點(合併飲品)',
 '異國料理',
 '小吃',
 '小吃',
 '日式',
 '小吃',
 '小吃',
 '甜點(合併飲品)',
 '飲品',
 '甜點(合併飲品)',
 '小吃',
 '甜點(合併飲品)',
 '異國料理',
 '異國料理',
 '異國料理',
 '甜點(合併飲品)',
 '飲品',
 '異國料理',
 '日式',
 '異國料理',
 '甜點(合併飲品)',
 '小吃',
 '小吃',
 '飲品',
 '甜點(合併飲品)',
 '小吃',
 '小吃',
 '異國料理',
 '小吃',
 '甜點(合併飲品)',
 '小吃',
 '甜點(合併飲品)',
 '甜點(合併飲品)',
 '小吃',
 '異國料理',
 '甜點(合併飲品)',
 '小吃',
 '異國料理',
 '小吃',
 '異國料理',
 '飲品',
 '小吃',
 '異國料理',
 '異國料理',
 '異國料理',
 '甜點(合併飲品)',
 '小吃',
 '甜點(合併飲品)',
 '異國料理',
 '小吃',
 '飲品',
 '甜點(合併飲品)',
 '小吃',
 '異國料理',
 '早午餐',
 '異國料理',
 '小吃',
 '飲品',
 '飲品',
 '小吃',
 '甜點(合併飲品)',
 '小吃',
 '日式',
 '飲品',
 '飲品',
 '小吃',
 '鍋類',
 '飲品',
 '小吃',
 '小吃',
 '小吃',
 '異國料理',
 '早午餐',
 '小吃',
 '鍋類',
 '甜點(合併飲品)',
 '甜點(合併飲品)',
 '甜點(合併飲品)',
 '飲品',
 '甜點(合併飲品)',
 '早午餐',
 '異國料理',
 '甜點(合併飲品)',
 '甜點(合併飲

In [51]:
#新增opentime closetime
for time,dien in zip(dienopen,bigtable):
    dien['opentime']=time[0]
    dien['closetime']=time[1]

In [52]:
[(dien['opentime'],dien['closetime']) for dien in bigtable]

[('09:00', '23:00'),
 ('08:00', '22:00'),
 ('05:00', '12:00'),
 ('06:00', '16:00'),
 ('11:00', '21:00'),
 ('11:00', '14:00'),
 ('11:30', '14:30'),
 ('11:00', '23:00'),
 ('08:00', '21:00'),
 ('07:00', '19:30'),
 ('11:00', '19:00'),
 ('00:00', '02:00'),
 ('08:00', '18:00'),
 ('11:00', '19:00'),
 ('11:00', '19:00'),
 ('11:00', '15:00'),
 ('10:00', '22:00'),
 ('12:00', '19:00'),
 ('11:00', '19:00'),
 ('11:00', '15:00'),
 ('11:00', '21:00'),
 ('08:30', '22:00'),
 ('11:00', '19:00'),
 ('16:30', '02:00'),
 ('11:00', '22:00'),
 ('10:00', '21:00'),
 ('11:00', '19:00'),
 ('09:00', '18:00'),
 ('11:00', '18:00'),
 ('00:00', '01:00'),
 ('12:00', '15:00'),
 ('07:30', '18:00'),
 ('07:40', '22:30'),
 ('10:00', '19:00'),
 ('10:30', '22:00'),
 ('11:00', '19:00'),
 ('06:00', '22:00'),
 ('13:00', '21:00'),
 ('11:00', '18:00'),
 ('09:00', '17:00'),
 ('10:00', '23:00'),
 ('09:00', '21:00'),
 ('11:00', '22:00'),
 ('09:30', '13:30'),
 ('11:00', '21:00'),
 ('11:00', '23:00'),
 ('10:00', '21:00'),
 ('00:00', '0

In [53]:
bigtable[0]

{'Ncomment': 26,
 'TF_IDF': {'一下': 0.0005058619310472912,
  '一個': 0.0006763427417325443,
  '一口': 0.000902388368012658,
  '一家': 0.0008383533159246719,
  '一張': 0.0009772699783033602,
  '一杯': 0.0007763227846572202,
  '一次': 0.0007700843326304692,
  '一碗': 0.0031732457338551666,
  '一種': 0.0006668078310359679,
  '一起': 0.0008712465982758182,
  '一間': 0.0008375586857847692,
  '一點': 0.0010014197611782284,
  '三種': 0.0007760097055008344,
  '上次': 0.0021988574511825607,
  '上面': 0.000604514746360345,
  '下': 0.00012088803629924028,
  '下次': 0.0006209773348036758,
  '不到': 0.0004732557107079696,
  '不用': 0.0006399935237685169,
  '不錯': 0.0003006029692292721,
  '中午': 0.0007751981504600589,
  '之後': 0.0007806238562467455,
  '二樓': 0.0011710565380702723,
  '享用': 0.0006151714242347759,
  '人': 0.0009087926521433939,
  '人潮': 0.0024724144797953255,
  '今天': 0.00044843015836489036,
  '介紹': 0.000613901062466487,
  '以上': 0.0007279636511091276,
  '以前': 0.0009350987128488293,
  '位置': 0.0010154508997988877,
  '來到': 0.00317

In [54]:
for rdien,adien in zip(rawdata,bigtable):
    adien['address']=rdien['address']
    adien['tel']=rdien['tel']

In [55]:
d='tags'
for dien in bigtable:
    if '麵.飯' in dien[d]:
        print(dien[d])
        dien[d].remove('麵.飯')
        print(dien[d])
        
d='TF_IDF'
for dien in bigtable:
    if '麵.飯' in dien[d]:
        print(dien[d])
        del dien[d]['麵.飯']
        print(dien[d])

['湯和魯味', '聽見', '麵.飯', '肉羹米粉', '搭車', '肉圓']
['湯和魯味', '聽見', '肉羹米粉', '搭車', '肉圓']
['中式早餐', '麵.飯', '點裡面', '湯最', '當歸', '中式']
['中式早餐', '點裡面', '湯最', '當歸', '中式']
{'吃': 0.003420683729196565, '咖': 0.008821895598846191, '搭車': 0.03450420188440814, '元': 0.004876191903958942, '四神湯': 0.022631047061808615, '聽見': 0.028849650690164676, '想': 0.0021766552737658017, '嫌': 0.014645321059193155, '應該': 0.004389330703872933, '不錯': 0.0033958359816591357, '回家': 0.008731877078674937, '王子': 0.024682718748375257, '約會': 0.017997539997291253, '一邊': 0.011936319085403956, '買': 0.004924240912863575, '夏日': 0.017676150456541216, '蜜': 0.008838070040609207, '愛喝': 0.017736250384345505, '水果': 0.008803914235725494, '香菇': 0.00681544547738303, '肉羹': 0.011852269872283609, '麵.飯': 0.024894758108742716, '台南市': 0.001287989685293627, '北區': 0.007411429576540433, '育德路': 0.020181070331370274, '號點': 0.023706725298350537, '餐時': 0.01092995727132595, '說': 0.0010775133881646324, '肉圓': 0.009301015533255881, '時常': 0.010356803871822652, '客人': 0.002

In [56]:
bigtable[0].keys()

dict_keys(['name', 'style', 'Ncomment', 'contentcut', 'wordCount', 'TF_IDF', 'pinCountMaxper5', 'score', 'featurescores', 'tags', 'introduction', 'coordinate', 'opentime', 'closetime', 'address', 'tel'])

In [57]:
gooddien=[dien for dien in bigtable if dien["score"]>30 and dien["Ncomment"]>4]

In [3]:
#登入pymongo
import pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [4]:
#確認裡面有哪幾個database
clinet.database_names()

['admin',
 'iii-2017-07',
 'local',
 'mydb',
 'project',
 'rawData',
 'test',
 'test-database']

In [5]:
#選擇rawData這個db
db=clinet['rawData']

In [6]:
#確認裡面有哪幾個collection
db.collection_names()

['mobile01_tainan',
 'Tainan_final',
 'TainanUnique',
 'merge_tainanUni',
 'Tbike',
 'ptt_tainan_address',
 'AipinwanFood',
 'ptt_tainan_tel',
 'Tainan_final_name_id_1',
 'pixnet_tainan_author',
 'FinishTainanBigTable',
 'ptt_tainan']

In [7]:
collection = db.FinishTainanBigTable

In [8]:
# collection.insert_many(bigtable)

In [9]:
# collection.drop()

In [10]:
x=collection.find({})

In [40]:
import pprint
pprint.pprint(x[0].keys())

dict_keys(['_id', 'name', 'style', 'Ncomment', 'contentcut', 'wordCount', 'TF_IDF', 'pinCountMaxper5', 'score', 'featurescores', 'tags', 'introduction', 'coordinate', 'opentime', 'closetime', 'address', 'tel'])


In [16]:
from collections import Counter
Counter(x[600]['TF_IDF']).most_common(30)

[('神農', 0.02950199290572356),
 ('閃電俠', 0.012215164499440235),
 ('天井', 0.011131793731209033),
 ('綠巨人', 0.008505402249309235),
 ('老宅', 0.008282264065970645),
 ('蒼蠅', 0.008262714780107367),
 ('蛋糕', 0.008109173102528188),
 ('是蟬', 0.007598030811538127),
 ('街', 0.007249720631081536),
 ('歇個', 0.006989889406156346),
 ('改造', 0.006651416443400486),
 ('木耳', 0.006433772102192859),
 ('浩克', 0.006275239914190481),
 ('太古', 0.005923773568871167),
 ('兩旁', 0.005872845308275393),
 ('國外', 0.005815307285283599),
 ('二樓', 0.0057526673825962155),
 ('藝廊', 0.005750613307851262),
 ('牠', 0.005340171951034787),
 ('飲料', 0.004934439798995943),
 ('斑馬', 0.0049268807211665724),
 ('角落', 0.0046830347326004725),
 ('看見', 0.004654728201697158),
 ('一尊', 0.004613947468670121),
 ('拍照', 0.004471845529878069),
 ('街頭', 0.0044374547316661855),
 ('前半部', 0.004258207551385917),
 ('布朗', 0.004184284878787171),
 ('廁所', 0.0041789819906456575),
 ('老闆', 0.004151944898175001)]

In [58]:
# import json
# with open('D:/Data/JsonData/TainanFood/FinishTainanBigTable.json','w') as f:
#     json.dump(bigtable,f)


In [59]:
# import json
# with open('D:/Data/JsonData/TainanFood/gooddien.json','w') as f:
#     json.dump(gooddien,f)

In [1]:
import json
with open('D:/Data/JsonData/TainanFood/FinishTainanBigTable.json') as f:
    xx=json.load(f)

In [2]:
xx[0]

{'Ncomment': 26,
 'TF_IDF': {'一下': 0.0005058619310472912,
  '一個': 0.0006763427417325443,
  '一口': 0.000902388368012658,
  '一家': 0.0008383533159246719,
  '一張': 0.0009772699783033602,
  '一杯': 0.0007763227846572202,
  '一次': 0.0007700843326304692,
  '一碗': 0.0031732457338551666,
  '一種': 0.0006668078310359679,
  '一起': 0.0008712465982758182,
  '一間': 0.0008375586857847692,
  '一點': 0.0010014197611782284,
  '三種': 0.0007760097055008344,
  '上次': 0.0021988574511825607,
  '上面': 0.000604514746360345,
  '下': 0.00012088803629924028,
  '下次': 0.0006209773348036758,
  '不到': 0.0004732557107079696,
  '不用': 0.0006399935237685169,
  '不錯': 0.0003006029692292721,
  '中午': 0.0007751981504600589,
  '之後': 0.0007806238562467455,
  '二樓': 0.0011710565380702723,
  '享用': 0.0006151714242347759,
  '人': 0.0009087926521433939,
  '人潮': 0.0024724144797953255,
  '今天': 0.00044843015836489036,
  '介紹': 0.000613901062466487,
  '以上': 0.0007279636511091276,
  '以前': 0.0009350987128488293,
  '位置': 0.0010154508997988877,
  '來到': 0.00317

# 加入Tbike資料進mongo

In [2]:
import json
with open('D:\Data\JsonData\Tbike\Fri_Sep__8-2.json') as f:
    Tbikedatalist=json.load(f)

In [3]:
Tbikedata=[{'Address':spot['Address'],'Id':spot['Id'],'Latitude':spot['Latitude'], 'Longitude':spot[ 'Longitude'],'StationName':spot['StationName']} for spot in Tbikedatalist[0]]

In [22]:
collection = db.Tbike
# collection.insert_many(Tbikedata)

In [23]:
collection = db.Tbike

In [24]:
collection.find_one()

{'Address': '保安轉運站公車侯車亭旁 (文賢路一段)',
 'Id': 3,
 'Latitude': 22.932706,
 'Longitude': 120.230637,
 'StationName': '保安轉運站',
 '_id': ObjectId('59c32f11aadaed1b6c70adf4')}

In [16]:
rawdata[0]['comment'][0]['content']

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/'

In [25]:
text='''簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域\
，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼\
降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光\
是在安平就有兩個店面'''
text

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面'

In [26]:
import jieba
" ".join(jieba.cut(text))

'簡單 、 傳統 的 安平 豆花 ， 純白 的 豆花 特別 的 挑逗 我們 的 胃口 。 安平 ， 兼具 了 台南 傳統 與 現代 的 一個 區域 ， 而作 為 安平 代表 的 傳統 美食 ， 一如 我們 所知 的 就是 安平 豆花 與 周氏 蝦 捲 了 。 今天 ， 帶點 炎熱 的 天氣 ， 是 來點 清涼 降火 冰品 的 好日子 ， 當然 是 來 喊 個 豆花 消暑 了 ， 而 吃 安平 豆花 當然 是 要 去 最 著名 的 同記 安平 豆花 。 同記 安平 豆花 光是在 安平 就 有 兩個 店面'

In [27]:
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        

In [28]:
words = jieba.cut(text, cut_all=False)
pincut=" ".join([word for word in words if word not in stopwordset and
                 '\u4e00' <= word and word <= '\u9fff' ])#and word not in dienlist and word not in pinlist
pincut

'簡單 傳統 安平 豆花 純白 豆花 特別 挑逗 胃口 安平 兼具 台南 傳統 現代 一個 區域 而作 安平 代表 傳統 美食 一如 所知 安平 豆花 周氏 蝦 捲 今天 帶點 炎熱 天氣 來點 清涼 降火 冰品 好日子 喊 豆花 消暑 吃 安平 豆花 最 著名 同記 安平 豆花 同記 安平 豆花 光是在 安平 兩個 店面'